In [1]:
#---Importing libraries---
import pandas as pd
import numpy as np
import time
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder,OrdinalEncoder

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# File location and type
df = pd.read_csv('/content/drive/MyDrive/Colab/supply_risk/clip_2.csv')
df.head(5)

,Vendor ID,Plant,MATNR,Material,Requested Delv Date,Requested Delv MM-YYYY,Requested Delv Week,Actual Delv Date,Actual Delv MM-YYYY,Actual Delv Week,...,VGU Age,VendorCountry,Avg Delv Date diff,OTIF,QTY Requested,Qty Overdelivered,QTY Received,Clip Score,SCAR Risk,Delivery status
0,100052427,BR02,3.000020e+11,Others,4/19/2022,22-Apr,17,4/19/2022,22-Apr,17,...,5,BR,0.0,1.0,350.0,0.0,350.0,1.0,1,0
1,100052427,BR02,3.000020e+11,Others,5/5/2022,22-May,19,5/5/2022,22-May,19,...,5,BR,0.0,1.0,1500.0,0.0,1500.0,1.0,1,0
2,100052427,BR02,3.000020e+11,Others,6/3/2022,22-Jun,23,4/19/2022,22-Apr,17,...,5,BR,-45.0,0.0,750.0,0.0,22.0,0.0,1,1
3,100052427,BR02,3.000020e+11,Others,6/21/2022,22-Jun,26,5/5/2022,22-May,19,...,5,BR,-47.0,0.0,750.0,0.0,92.0,0.0,1,1
4,100052427,BR02,3.000040e+11,Others,4/29/2022,22-Apr,18,4/28/2022,22-Apr,18,...,5,BR,-1.0,1.0,150.0,0.0,150.0,1.0,1,0


In [27]:
df.columns

Index(['Vendor ID', 'Plant', 'MATNR', 'Material', 'Requested Delv Date',
       'Requested Delv MM-YYYY', 'Requested Delv Week', 'Actual Delv Date',
       'Actual Delv MM-YYYY', 'Actual Delv Week', 'Breadman Y/N', 'SQOPE500',
       'VGU Age', 'VendorCountry', 'Avg Delv Date diff', 'OTIF',
       'QTY Requested', 'Qty Overdelivered', 'QTY Received', 'Clip Score',
       'SCAR Risk', 'Delivery status'],
      dtype='object')

# Clubbing low priority materials

In [28]:
pred_df3 = df[['Vendor ID', 'Plant', 'Material', 'Requested Delv Week','SQOPE500',
       'VGU Age', 'VendorCountry','Clip Score','QTY Requested','SCAR Risk', 'Delivery status']]

In [19]:
pred_df3.head(5)

,Vendor ID,Plant,Material,Requested Delv Week,SQOPE500,VGU Age,VendorCountry,Clip Score,QTY Requested,SCAR Risk,Delivery status
0,100052427,BR02,Others,17,Rest,5,BR,1.0,350.0,1,0
1,100052427,BR02,Others,19,Rest,5,BR,1.0,1500.0,1,0
2,100052427,BR02,Others,23,Rest,5,BR,0.0,750.0,1,1
3,100052427,BR02,Others,26,Rest,5,BR,0.0,750.0,1,1
4,100052427,BR02,Others,18,Rest,5,BR,1.0,150.0,1,0


In [29]:
#Preprocessing
pred_df3.loc[pred_df3["VGU Age"] == "(blank)", "VGU Age"] = 0
pred_df3.loc[pred_df3["SQOPE500"] == "Rest", "SQOPE500"] = 0
pred_df3.loc[pred_df3["SQOPE500"] == "Top500", "SQOPE500"] = 1
pred_df3['Requested Delv Week'] = pred_df3['Requested Delv Week'].astype('int')
pred_df3['VGU Age'] = pred_df3['VGU Age'].astype('int')
pred_df3['QTY Requested'] = pred_df3['QTY Requested'].astype('float')
pred_df3['SCAR Risk'] = pred_df3['SCAR Risk'].astype('int')

In [30]:
pred_df3 = pred_df3[pred_df3['Material'] != "Others"]
pred_df3.head(5)

,Vendor ID,Plant,Material,Requested Delv Week,SQOPE500,VGU Age,VendorCountry,Clip Score,QTY Requested,SCAR Risk,Delivery status
16,100052427,BR02,3.00006E+11,50,0,5,BR,1.0,100.0,1,0
17,100052427,BR02,3.00006E+11,6,0,5,BR,1.0,1500.0,1,0
18,100052427,BR02,3.00006E+11,9,0,5,BR,0.0,1000.0,1,1
19,100052427,BR02,3.00006E+11,11,0,5,BR,1.0,2000.0,1,0
20,100052427,BR02,3.00006E+11,12,0,5,BR,1.0,2000.0,1,0


In [12]:
pred_df3['Material'].nunique()

117

In [31]:
X = pred_df3.drop(['Delivery status'], axis=1)
y = pred_df3['Delivery status']
X = X.drop(['Vendor ID'], axis = 1)
cat=['Plant','SQOPE500','VendorCountry', 'Material']
for i in cat:
    pred_df3[i] = pred_df3[i].astype('category')
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, cat)])

In [54]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [55]:
# Logistic Regression
lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('lr', LogisticRegression())])
lr.fit(X_train, y_train)
print("model Train score: %.3f" % lr.score(X_train, y_train))
print("model Test score: %.3f" % lr.score(X_test, y_test))

model Train score: 0.867
model Test score: 0.868


In [56]:
y_pred=lr.predict(X_test)
cnf_matrix = confusion_matrix(np.array(y_test).astype('int'), y_pred.astype(int), labels=[1,0])
cnf_matrix

array([[ 3191,  9852],
       [ 1625, 72335]])

In [57]:
# Random Forest classification
rfc = Pipeline(steps=[('preprocessor', preprocessor),
                     ('rf', RandomForestClassifier(random_state=0))])
rfc.fit(X_train, y_train)
print("model Train score: %.3f" % rfc.score(X_train, y_train))
print("model Test score: %.3f" % rfc.score(X_test, y_test))

model Train score: 0.871
model Test score: 0.872


In [58]:
y_pred=rfc.predict(X_test)
cnf_matrix = confusion_matrix(np.array(y_test).astype('int'), y_pred.astype(int), labels=[1,0])
cnf_matrix

array([[ 3749,  9294],
       [ 1884, 72076]])

In [59]:
# XGB classification
xgb = Pipeline(steps=[('preprocessor', preprocessor),
                     ('xg', xgb.XGBClassifier())])
xgb.fit(X_train, y_train)
print("model Train score: %.3f" % xgb.score(X_train, y_train))
print("model Test score: %.3f" % xgb.score(X_test, y_test))

model Train score: 0.871
model Test score: 0.872


In [60]:
y_pred=xgb.predict(X_test)
cnf_matrix = confusion_matrix(np.array(y_test).astype('int'), y_pred.astype(int), labels=[1,0])
cnf_matrix

array([[ 3740,  9303],
       [ 1862, 72098]])

In [50]:
# Linear SVC classification
sv = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sv', LinearSVC())])
sv.fit(X_train, y_train)
print("model Train score: %.3f" % sv.score(X_train, y_train))
print("model Test score: %.3f" % sv.score(X_test, y_test))

model Train score: 0.867
model Test score: 0.868


In [53]:
y_pred=sv.predict(X_test)
cnf_matrix = confusion_matrix(np.array(y_test).astype('int'), y_pred.astype(int), labels=[1,0])
cnf_matrix

array([[ 3076,  9967],
       [ 1506, 72454]])